In [ ]:
#Zaimportowanie wymaganych bibliotek
import os
import numpy as np
from pathlib import Path
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, Conv2DTranspose
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from numpy import zeros,ones,vstack
from numpy.random import randn,randint,normal
from numpy.random import default_rng
from IPython.display import clear_output
from math import trunc

from matplotlib import pyplot

from PIL import Image

In [ ]:
#Parametry
VERSION=0.7
#PATH_TO_DATA='data/cubism_data_norm.npy'
#DATA="cubism"
PATH_TO_DATA='data/delaunay_data_norm.npy'
DATA="delaunay"

#Number of saved images during training
OUTPUT = 8
OUTPUT_MARGIN = 4
SAVE_FREQ = 100 #How often to save images
OUTPUT_PATH='output\\new'
'''
In this use case, our latent space representations are used to
transform more complex forms of raw data (i.e. images, video),
into simpler representations which are "more convenient to process" and analyze.
'''
NOISE_SIZE = 128#Lantent dimention size

EPOCHS = 5000 #Iterations 100000 the biger, the longer the model will train
BATCH_SIZE = 64 #number of images in the Batch. Larger Batch → Weak Generalization, Larger Batches → Fewer updates + shifting data → lower computational costs

AVERAGE=["Epoch","Accuracy on real","Accuracy on fake"]
LOSS=["Epoch","Batch/Batch","Dyskryminator loss","Generator Loss"]


In [ ]:
train_data = np.load(PATH_TO_DATA)
print("(number of images,size_x,size_y,color_channels)")
print(train_data.shape)
if train_data.shape[1]!=train_data.shape[2]:
    print("Something is wrong")

In [ ]:
DATASET_NAME="cub"
if DATA =="delaunay":
    DATASET_NAME="del"


In [ ]:
#size of images
IMG_SIZE = train_data.shape[1] #rows/cols
IMG_CHANNELS = train_data.shape[3] #color channels in our images

In [ ]:
Image.fromarray(((train_data[1]+1)*255/2).astype(np.uint8))

In [ ]:
#discriminator
def build_discriminator(image_shape):
    model = Sequential(name="Discriminator")
        
    #model.add(Conv2D(32, kernel_size=4, strides=2,input_shape=image_shape,data_format="channels_last",padding="same"))
    #model.add(BatchNormalization(axis=1,momentum=0.8))
    #model.add(LeakyReLU(alpha=0.2))
    #model.add(Dropout(0.25))#dropout layer to prevent overfitting    
    
    model.add(Conv2D (64, kernel_size=4, strides=2, padding="same",input_shape=image_shape,data_format="channels_last"))
    #model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.20))    

    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.20))    
    
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.20))
    
    model.add(Conv2D(256, kernel_size=4, strides=1, padding="valid"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))    
    model.add(Dropout(0.20))
    
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))#binary clasification

    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=binary_crossentropy,optimizer=opt,metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
#generator
def build_generator(noise_size, channels):
     model = Sequential(name="Generator")
     model.add(Dense(1 * 1 * 128, activation="relu",input_dim=noise_size))
     model.add(Reshape((1, 1, 128)))    
     model.add(Conv2DTranspose(512,kernel_size=4, strides=1, padding="valid"))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))    
               
     #model.add(UpSampling2D())
     #model.add(Conv2D(256, kernel_size=4, padding="same"))
     model.add(Conv2DTranspose(256,kernel_size=4, strides=2, padding="same"))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))    

     #model.add(UpSampling2D())
     #model.add(Conv2D(256, kernel_size=4, padding="same"))
     model.add(Conv2DTranspose(128,kernel_size=4, strides=2, padding="same"))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))
     
     model.add(Conv2DTranspose(64,kernel_size=4, strides=2, padding="same"))
     model.add(BatchNormalization(momentum=0.8))
     model.add(Activation("relu"))        
          
     model.add(Conv2DTranspose(channels, kernel_size=4,strides=2, padding="same"))
     model.add(Activation("tanh"))    

     model.summary()

     return model

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential(name="GAN")
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    model.summary()
    return model

In [ ]:
def save_images(generator, noise, number):
    generated_images = generator.predict(noise)
    array=np.empty(((IMG_SIZE,IMG_SIZE,IMG_CHANNELS)),float)

    #print(generated_images)
    #from (-1,1) to (0,255)
    #generated_images=np.array(generated_images.reshape(OUTPUT*IMG_SIZE,OUTPUT*IMG_SIZE,IMG_CHANNELS), dtype=np.uint8)
    generated_images = (generated_images+1)/2*255
    count=0
    for i in range(y):
        for j in range(x):
            if j==0:
                array_pom=np.array(X[count].reshape(64,64,3,order='C'))
            else:
                array_pom=np.concatenate((array_pom,np.array(X[count].reshape(64,64,3,order='C'))),axis=1)
            count+=1
        if i==0:
            array=array_pom
        else:
            array=np.concatenate((array,array_pom),axis=0)
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    filename = os.path.join(OUTPUT_PATH, DATA+"-%d.png"%(number))
    im = Image.fromarray(((array+1)*255/2).astype(np.uint8))
    im.save(filename)

In [ ]:
def get_real_samples(data, n_samples):
    indexes = randint(0, data.shape[0], n_samples)
    X = data[indexes]
    y = ones((n_samples, 1))
    return X, y

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = default_rng().normal(0.0, 1.0, (n_samples, latent_dim))
    X = generator.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y


In [ ]:
def summarize_performance(epoch, generator, discriminator, dataset, latent_dim, n_samples=100):
    #getting samples and evaluating
    X_real, y_real = get_real_samples(dataset, n_samples)
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    AVERAGE.append([epoch,acc_real*100,acc_fake*100])
    print('Accuracy on real: %.0f%%, on fake: %.0f%%' % (acc_real*100, acc_fake*100))
    filename = DATASET_NAME+'_model_v_%.1f_e_%d.h5' % (VERSION,epoch + 1)
    generator.save('models\\new\\'+filename,include_optimizer=True)
    return acc_real,acc_fake

In [ ]:
TEMP_ROUNDS=0
TEMP_ROUNDS_2=0
def check_conditions(accuracy_real,accuracy_fake):
    #if both accuraccies is 100% for 5 epochs, than end program
    condition_1=round(accuracy_real,2)==100.00 and round(accuracy_fake,2)==100.00
    #Generator will be fulling discriminator with 100% accuracy for 10 epochs
    condition_2=round(accuracy_fake,2)==100.00
    if condition_1:
        TEMP_ROUNDS+=1
    else:
        TEMP_ROUNDS=0
    if condition_2:
        TEMP_ROUNDS_2+=1
    else:
        TEMP_ROUNDS_2=0
    if TEMP_ROUNDS==5 or TEMP_ROUNDS_2==10:
        return False 
    return True 

In [ ]:
def save_output(avg=AVERAGE,loss=LOSS,ver=VERSION,dataset=DATASET_NAME):
    np.savetxt("stat_avg_"+dataset+"_%d.csv"%(ver), avg, delimiter =", ", fmt ='% s')
    np.savetxt("stat_loss_"+dataset+"_%d.csv"%(ver), loss, delimiter =", ", fmt ='% s')
    

In [ ]:
def train(generator, discriminator, gan_model, dataset, 
          latent_dim, n_epochs=100, n_batch=10):
    batchs_per_epoch = int(dataset.shape[0] / n_batch)
    half_of_batch=int(int(n_batch / 2))
    #generating fixed noise for examples images
    fixed_noise = default_rng().normal(
        loc=0.0, scale=1.0, size=(OUTPUT * OUTPUT, latent_dim))
    number=1
    i=0
    work=True
    while i <= n_epochs and work:
        for j in range(batchs_per_epoch):
            #geting and generating samples
            X_real, y_real = get_real_samples(dataset, half_of_batch)
            X_fake, y_fake = generate_fake_samples(
                generator, latent_dim, half_of_batch)
            #joining the real images with false ones
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            X_gan = default_rng().normal(0.0, 1.0, (n_batch,latent_dim))
            y_gan = ones((n_batch, 1))
            #training
            d_loss,_ = discriminator.train_on_batch(X, y)
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            LOSS.append([i+1,j+1,d_loss, g_loss])
            print('Epoch:%d, Batch:%d/%d, d_loss=%.4f, g_loss=%.4f' %
                  (i+1, j+1, batchs_per_epoch, d_loss, g_loss))
        if (i+1) % 10 == 0:
            #saving images based on fixed noise
            save_images(generator, fixed_noise, number)
            save_output()
            number+=1
            clear_output()
            #getting accuracy
            acc_real,acc_fake=summarize_performance(
                i, generator, discriminator, dataset, latent_dim)
            #checking the conditions to stop
            work=check_conditions(acc_real,acc_fake)
        i+=1
            

In [ ]:
image_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
discriminator = build_discriminator(image_shape)
generator = build_generator(NOISE_SIZE, IMG_CHANNELS)

gan_model = build_gan(generator, discriminator)
#print(train_data.shape)
train(generator, discriminator, gan_model, np.array(train_data), NOISE_SIZE,EPOCHS)
#print(train_data)

In [ ]:
def get_gen_model(filename=""):
    generator=Sequential()
    if(filename!=""):
        generator=load_model("models\\"+filename,compile=False)
    return generator

model = get_gen_model("gen_model_v_0.3_e_63.h5")
x=5
y=5
latent_points = default_rng().normal(0.0, 1.0, (x*y, NOISE_SIZE))
image = Image.fromarray(((latent_points+1)*255/2).astype(np.uint8))
image.show()
X = model.predict(latent_points)
#array = np.array(((0.5*X.reshape(64,64,3)+1)*255).astype(np.uint8))
array=np.empty((IMG_SIZE,IMG_SIZE,IMG_CHANNELS),float)
count=0
for i in range(y):
    for j in range(x):
        if j==0:
            array_pom=np.array(X[count].reshape(64,64,3,order='C'))
        else:
            array_pom=np.concatenate((array_pom,np.array(X[count].reshape(64,64,3,order='C'))),axis=1)
        count+=1
    if i==0:
        array=array_pom
    else:
        array=np.concatenate((array,array_pom),axis=0)
#array = np.array(((X.reshape(x*64,64,3,order='A')+1)*255/2).astype(np.uint8))

image = Image.fromarray(((array+1)*255/2).astype(np.uint8))
image.show()
#image.save("output\\a1.png")